In [1]:
from search_engine.search_engine import SearchEngine
import networkx as nx
from networkx import Graph
from sklearn.model_selection import train_test_split
import implicit
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import igraph
import warnings
import csv
warnings.filterwarnings('ignore')

In [2]:
search_engine = SearchEngine()

Collection time: 0.0


In [3]:
search_engine.tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
created_at                 10000 non-null object
id_str                     10000 non-null object
text                       10000 non-null object
source                     10000 non-null object
truncated                  10000 non-null bool
in_reply_to_screen_name    1522 non-null object
user                       10000 non-null object
retweeted_status           6972 non-null object
reply_count                10000 non-null int64
retweet_count              10000 non-null int64
favorite_count             10000 non-null int64
entities                   10000 non-null object
favorited                  10000 non-null bool
retweeted                  10000 non-null bool
filter_level               10000 non-null object
timestamp_ms               10000 non-null object
extended_tweet             1451 non-null object
extended_entities          441 non-null object
withheld_in_countries   

In [4]:
#u -> search_engine.tweets.user[0]['screen_name']
#v -> search_engine.tweets.retweeted_status[1]['user']['screen_name']

In [5]:
#Para Poner bonito -> dictionario de los usuarios v
#Mirar página en favoritos, lo del color también en la NS_P09
#Add weight edges from how many times user u retweeted user v

#g = nx.Graph()
#for tweet in search_engine.tweets.iterrows():
#    if str(tweet[1]['retweeted_status'])!='nan':
#        u=tweet[1]['user']['screen_name']
#        v=tweet[1]['retweeted_status']['user']['screen_name']
#        g.add_node(u)
#        g.add_node(v)
#        g.add_edge(u,v)

g=igraph.Graph()
for tweet in search_engine.tweets.iterrows():
    if str(tweet[1]['retweeted_status'])!='nan':
        u=tweet[1]['user']['screen_name']
        v=tweet[1]['retweeted_status']['user']['screen_name']
        g.add_vertices(u)
        g.add_vertices(v)
        g.add_edges([(u,v)])

In [6]:
#pos=nx.spring_layout(g)
#plt.figure(figsize=(15,15))
#nx.draw_networkx(g, pos, node_shape='o', node_size=50, with_labels=False, font_weight='bold')
#plt.show()

#layout = g.layout("kk")
#igraph.plot(g, layout = layout, target="/other-outputs/igraph.png")

*Recomender Systems*

In [282]:
#SELCET USER ID:
user_id=0

In [283]:
def find_nodes_at_distance_2(graph):

    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(n1, order=1))

        # all the nodes at distance 1 and distance 2

        nodes_at_most_distant_2 = set(graph.neighborhood(n1, order=2))

        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        
        # check if empty set
        if len(nodes_at_most_distant_2) > 0:
            for n2 in nodes_at_most_distant_2:
                
                # since n1 is an igraph vertex object, we need to extract the id
                n1_index = n1.index
                if n1_index!=n2:
                    all_potential_recommendations.add((n1_index, n2))
            
    return all_potential_recommendations

def predict_ALS(testset, model):
    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1,n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [284]:
edges_max_distance_2=find_nodes_at_distance_2(g)

# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(g.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N),replace=False)

ALS (Alternate Least Squares) #VER SI SON LOS FRIENDS-OF-FRIENDS

In [285]:
def number_to_username(alg_list: list, g) -> list:
    alg_list_transformed=[]
    for i in range(len(alg_list)):
        name=g.vs[int(alg_list[i][0])]['name']
        new_tuple=(name, alg_list[i][1])
        alg_list_transformed.append(new_tuple)
    return alg_list_transformed

def recommend_users(user_id: int, G:csr_matrix, g: igraph.Graph, top: int = 10) -> list:
    ALS_recommended_users=model.recommend(user_id, G, top)
    #ALS_recommended_users_with_names=number_to_username(ALS_recommended_users, g)
    return ALS_recommended_users

In [286]:
ground_truth = set()
trainset = set()
for idx, one_edge in enumerate(g.es):
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target
    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
    else:
        trainset.add((n1, n2, 1))
        
for rec in edges_max_distance_2:
    
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    
    ground_truth.add((n1,n2,0))

In [265]:
G = g.get_adjacency().data
G = csr_matrix(G)

In [287]:
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=5, calculate_training_loss=True)
model.fit(G)

In [288]:
# generate the predictions
df_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])

all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: round(x))

In [289]:
df_test.head()

,n1,n2,edge,rating
0,1123,1528,1,0
1,3896,1098,0,0
2,844,748,0,0
3,3788,10604,0,0
4,9976,844,0,0


In [290]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test.edge == df_test.rating])

# accuracy
right_predictions/len(df_test)

0.9890923907507385

In [304]:
ALS_recomdetaion_ids=recommend_users(user_id, G, g)
ALS_recomdetaion_names=number_to_username(ALS_recomdetaion_ids, g)
ALS_recomdetaion_ids

[(97, 0.09225066),
 (77, 0.0677746),
 (93, 0.048736416),
 (73, 0.04281413),
 (65, 0.0396551),
 (111, 0.032695185),
 (311, 0.0326447),
 (53, 0.03171517),
 (7, 0.030292317),
 (133, 0.029419735)]

DCG ALS

In [305]:
print(g.neighborhood(user_id, order=1))
print()
print(g.neighbors(2029))
print(g.neighbors(1))

[0, 1, 2029]

[0, 516, 1282, 3422, 4130, 4618, 5988, 7290, 8542, 10358, 10700]
[0, 112, 266, 482, 652, 914, 1055, 1282, 2168, 2214, 2504, 2866, 3022, 3180, 3210, 3534, 3744, 4174, 4340, 4754, 5418, 6014, 6534, 6902, 7164, 7276, 8188, 8228, 8322, 8434, 8990, 9014, 9068, 9206, 9300, 9660, 9792, 9954, 10484, 10982, 11224, 11468, 11484, 11742, 12316, 12420, 12458, 13436]


 ---------------------------------------------------------------------------------------------------------------------------

Adamic-Adar

In [142]:
def get_recommendation_AA(username: str, g:igraph.Graph) -> pd.DataFrame:
    x_neighbors=set()
    y_neighbors=set()
    usersu=[]
    usersv=[]
    for i in g.es:
        usersu.append(i.tuple[0])
        usersv.append(i.tuple[1])
    
    users_unique_list=set(usersu)|set(usersv)
    adamic_avar_data=pd.DataFrame(columns=[username], index=users_unique_list)
    for user_y in users_unique_list:
        if username!=user_y:
            x_neighbors=set(g.neighbors(username))
            y_neighbors=set(g.neighborhood(user_y))
        same_neighbors=x_neighbors&y_neighbors
        aa_val=0
        for n in same_neighbors:
            num_neighbors=len(g.neighbors(n))
            aa_val+=(1/math.log(num_neighbors,10))
        adamic_avar_data[username][user_y]=aa_val
    top_n_recommendations_aa=adamic_avar_data[username].sort_values(ascending=False)
    aa_final_recommendation=pd.DataFrame(top_n_recommendations_aa)
    return aa_final_recommendation.head(10)

def AA_num_to_name(dataset: pd.DataFrame, g: igraph.Graph)->pd.DataFrame:
    old_indices=list(dataset.index)
    new_indices=[]
    #name_column=g.vs[dataset=columns[0]]['name']
    main_user_id=dataset.columns[0]
    main_user_name=g.vs[main_user_id]['name']
    for user_id in old_indices:
        name=g.vs[user_id]['name']
        new_indices.append(name)
    new_dataset=pd.DataFrame(dataset.values, columns=[main_user_name], index=new_indices)
    return new_dataset

In [154]:
AA_recommendation_ids=get_recommendation_AA(user_id, g)
AA_recommendation_names=AA_num_to_name(AA_data, g)
AA_recommendation_ids

,2
8640,0.698634
11870,0.698634
554,0.698634
10854,0.698634
11648,0.698634
3960,0.698634
1210,0.698634
1608,0.698634
3422,0.698634
6522,0.698634


In [86]:
#users_u=[names[0] for names in list(g.edges)]
#users_v=[names[1] for names in list(g.edges)]
#Introduce a username to get its friends recommendations
#u='1Krohmer13'
#get_recommendation_AA(u, users_u, users_v)

----------------------------------------------------------------------------------------------------------------------------

PageRank

In [254]:
def pagerank_clearer(pagerank_values: list, g: igraph.Graph) -> list:
    pagerank=[]
    for i in range(len(pagerank_values)):
            user=pagerank_values[i][0]
            name=g.vs[user]['name']
            val=float(pagerank_values[i][1])
            pagerank.append((name, val))
    return pagerank

def top_10_ids(pagerank_result: list, user_id: int)->list:
    pagerank_with_ids=[]
    for i in range(len(pagerank_result)):
        if i!=user_id:
            pagerank_with_ids.append([i, pagerank_result[i]])
    pagerank_with_ids.sort(key = lambda x: x[1], reverse=True)
    pagerank_top_10_ids=pagerank_with_ids[0:10]
    return pagerank_top_10_ids

In [257]:
##This will give the values of the users normalized??
#Do list (id, punctuation)
PageRank_recommendations=g.personalized_pagerank(directed=True, reset_vertices=user_id)
PageRank_recommendation_ids=top_10_ids(PageRank_recommendations, user_id)
PageRank_recommendation_names=pagerank_clearer(PageRank_recommendation_ids, g)
PageRank_recommendation_names

[('aaronjcarpenter', 0.14464007260674197),
 ('GOPLeader', 0.10436465506455363),
 ('m_damuy', 0.01411323598099609),
 ('realDonaldTrump', 0.012462920530383611),
 ('kayleighmcenany', 0.011864371180461712),
 ('mhoran1158', 0.011798789754219368),
 ('Madnessauto69', 0.010989788472787218),
 ('EReldeonk', 0.009768140426121097),
 ('hallroland', 0.009089399533786615),
 ('ZieglerDanny', 0.008438049509322554)]

----------------------------------------------------------------------------------------------------------------------------

Node2vec

In [258]:
import argparse
import numpy as np
import networkx as nx
import node2vec
#from gensim.models import Word2Vec

In [259]:
A = g.get_edgelist()
graph = nx.Graph(A) # In case your graph is directed

In [260]:
#help(node2vec.Node2Vec)

In [261]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = node2vec.Node2Vec(graph, dimensions=64, walk_length=2, num_walks=200, workers=4) #Use temp_folder for big graphs
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, 
                                                             #`dimensions` and `workers` are automatically passed (from the 
                                                             #Node2Vec constructor)

In [262]:
user_id_str=str(user_id)
node2vec_recommendation_ids=model.wv.most_similar(user_id_str)

In [73]:
node2vec_recommendation_names=number_to_username(node2vec_recommendation_ids, g)

[('sherryclayton53', 0.9992790222167969),
 ('Miguelach19', 0.9992578029632568),
 ('PikachuEXE', 0.9992504119873047),
 ('CharmaneWargo1', 0.9991941452026367),
 ('AshleyW838', 0.9991587996482849),
 ('LuzSamiraRodrg2', 0.9991163611412048),
 ('goncen60', 0.9990949630737305),
 ('Pjxport3', 0.9642602801322937),
 ('herbySk', 0.94817054271698),
 ('ROB_ENGLAND_', 0.9449869394302368)]

----------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Definitions that were created for ALS

#def adjency_matrix(vocabulary: dict, edges: list) -> csr_matrix:
#    adj_matrix=pd.DataFrame(0, columns=list(vocabulary.values()), index=list(vocabulary.values()))
#    A=np.array(edges)
#    for i in range(len(A)):
#        A[i][0]=vocabulary[A[i][0]]
#        A[i][1]=vocabulary[A[i][1]]
#    for i in range(len(A)):
#        useru=int(A[i][0])
#        userv=int(A[i][1])
#        adj_matrix[useru][userv]=1
#        adj_matrix[userv][useru]=1
#        adj_matrix_sparse=sparse.csr_matrix(adj_matrix.values)
#    return adj_matrix_sparse

#def similar_users(user_id: int, top: int, vocabulary:dict) -> list:
#    ALS_similar_users=model.similar_users(user_id,top)
#    ALS_similar_users_with_names=number_to_username(ALS_similar_users, vocabulary)
#    return ALS_similar_users_with_names

#def recommend_users(user_id: int, G:csr_matrix, top: int, vocabulary:dict) -> list:
#    ALS_recommended_users=model.recommend(user_id, G, top)
#    ALS_recommended_users_with_names=number_to_username(ALS_recommended_users, vocabulary)
#    return ALS_recommended_users_with_names

#def number_to_username(ALS_list: list, vocabulary: dict) -> list:
#    ALS_list_transformed=[]
#    for i in range(len(ALS_list)):
#        name=list(vocabulary.keys())[list(vocabulary.values()).index(ALS_list[i][0])]
#        new_tuple=(name, ALS_list[i][1])
#        ALS_list_transformed.append(new_tuple)
#    return ALS_list_transformed

In [ ]:
#Definitions that were created for A-A

#def get_neighbors(username: str, usersu: list, usersv: list) -> list:
#    neighbors=[]
#    for i in range(len(usersu)):
#        if (username==usersu[i]):
#            neighbors.append(usersv[i])
#        if (username==usersv[i]):
#            neighbors.append(usersu[i])
#    return neighbors

#def get_recommendation_AA(username: str, usersu: list, usersv: list) -> pd.DataFrame:
#    users_unique_list=set(usersu)|set(usersv)
#    adamic_avar_data=pd.DataFrame(columns=[username], index=users_unique_list)
#    for user_y in users_unique_list:
#        if username!=user_y:
#            x_neighbors=set(get_neighbors(username, usersu, usersv))
#            y_neighbors=set(get_neighbors(user_y, usersu, usersv))
#        same_neighbors=x_neighbors&y_neighbors
#        aa_val=0
#        for n in same_neighbors:
#            num_neighbors=len(get_neighbors(n, usersu, usersv))
#            aa_val+=(1/math.log(num_neighbors))
#        adamic_avar_data[username][user_y]=aa_val
#    top_n_recommendations_aa=adamic_avar_data[username].sort_values(ascending=False)
#    aa_final_recommendation=pd.DataFrame(top_n_recommendations_aa)
#    return aa_final_recommendation.head(10)